In [1]:
import pandas as pd
pois = pd.read_csv("raw_poi_data.csv")

In [2]:
pois.isnull().sum()

name          0
category      0
lat           0
lon           0
LocationID    0
zone          0
dtype: int64

In [3]:
pois = pois.rename(columns={'lat': 'latitude', 'lon': 'longitude', 'zone': 'zone_name', 'name': 'poi_name', 'category': 'poi_type_name', 'LocationID': 'zone_id'})
pois.drop('zone_name', axis=1, inplace=True)

pois['poi_type_name'] = pois['poi_type_name'].str.lower()
pois['poi_name'] = pois['poi_name'].str.lower()
# pois = pois.drop_duplicates(subset='poi_name', keep='first')
# pois can have same names for different poi types

In [4]:
import numpy as np
poi_type_names = sorted(pois['poi_type_name'].unique())
poi_type_ids = (np.arange(len(poi_type_names))+1).tolist()
poi_type = pd.DataFrame({'poi_type_id': poi_type_ids, 'poi_type_name': poi_type_names})
poi_type.to_csv('poitype_data.csv', index=False)

In [5]:
pois['poi_type_id'] = pois['poi_type_name'].map(poi_type.set_index('poi_type_name')['poi_type_id'])
pois.drop('poi_type_name', axis=1, inplace=True)
pois = pois.sort_values(by='poi_name', ascending=True)
# pois['poi_id'] = np.arange(1, len(pois)+1)
# assign poi_id automatically by database
pois.to_csv('poi_data.csv', index=False)

In [6]:
import fiona

with fiona.open("taxi_zones/taxi_zones.shp", "r") as src:
    features = [f for f in src]
    zones = pd.DataFrame([f['properties'] for f in features])

zones = zones[['LocationID', 'zone']]
zones = zones.rename(columns={'LocationID': 'zone_id', 'zone': 'zone_name'})
zones['zone_name'] = zones['zone_name'].str.lower()
zones = zones.drop_duplicates(subset='zone_name', keep='first')
zones.to_csv('zone_data.csv', index=False)

In [7]:
zones

,zone_id,zone_name
0,1,newark airport
1,2,jamaica bay
2,3,allerton/pelham gardens
3,4,alphabet city
4,5,arden heights
...,...,...
258,259,woodlawn/wakefield
259,260,woodside
260,261,world trade center
261,262,yorkville east


In [8]:
zones_id = zones['zone_id'].tolist()
pro = []
for i in range(1, 263):
    if (i not in zones_id):
        pro.append(i)
print(pro)

[57, 104, 105]


In [9]:
poi_type

,poi_type_id,poi_type_name
0,1,attraction
1,2,cafe
2,3,museum
3,4,park
4,5,pub
5,6,restaurant


In [10]:
pois

,poi_name,latitude,longitude,zone_id,poi_type_id
11342,"""u"" like chinese take out",40.867494,-73.921726,127.0,6
33995,"""uncle"" vito f. maranzano glendale playground",40.704932,-73.881757,102.0,4
19349,"""w"" cafe",40.749800,-73.984080,164.0,6
10803,#1 garden chinese restaurant,40.660156,-73.980555,257.0,6
1008,#1 natural juice bar,40.703061,-73.907862,198.0,2
...,...,...,...,...,...
18232,zz's pizza & grill,40.643726,-74.072789,206.0,6
2002,züri coffee,40.783593,-73.979832,239.0,2
507,épicerie boulud,40.711288,-74.010275,261.0,2
1845,épicerie boulud,40.711774,-74.011669,261.0,2


In [3]:
# include poi data with user_rating
import pandas as pd
pois = pd.read_csv("poi_data_ryan.csv")
pois.drop('num_of_reviews', axis=1, inplace=True)
pois.drop('price_level', axis=1, inplace=True)
pois.drop('gmap_id', axis=1, inplace=True)
pois.drop('source', axis=1, inplace=True)
pois.rename(columns={'avg_rating':'user_rating'}, inplace=True)
pois['picture_url'] = None
pois.to_csv("poi_data.csv", index=False)

In [4]:
pois

,poi_name,latitude,longitude,zone_id,poi_type_id,user_rating,picture_url
0,"""u"" like chinese take out",40.867494,-73.921726,127.0,6,NaN,None
1,"""Uncle"" Vito F. Maranzano Glendale Playground",40.704932,-73.881757,102.0,4,4.0,None
2,"""w"" cafe",40.749800,-73.984080,164.0,6,NaN,None
3,#1 garden chinese restaurant,40.660156,-73.980555,257.0,6,NaN,None
4,#1 natural juice bar,40.703061,-73.907862,198.0,2,NaN,None
...,...,...,...,...,...,...,...
34684,zz's pizza & grill,40.643726,-74.072789,206.0,6,NaN,None
34685,züri coffee,40.783593,-73.979832,239.0,2,NaN,None
34686,épicerie boulud,40.711288,-74.010275,261.0,2,NaN,None
34687,épicerie boulud,40.711774,-74.011669,261.0,2,NaN,None
